In [1]:
server_args = []
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

from manipulation import running_as_notebook

# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge, le, lt, gt, ne, AutoDiffXd, autoDiffToGradientMatrix, SnoptSolver,
    initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix,
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint,
    MeshcatContactVisualizer, Expression
)
from pydrake.symbolic import Variable

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

In [3]:
#add table
ground = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/ground.urdf"), "ground")

#add box
cracker_box = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box_test.urdf"), "cracker_box")


In [4]:
plant.Finalize()
plant.set_name("plant")

In [5]:
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())


draw_frames = True
frames_to_draw = [cracker_box_id] if draw_frames else []
meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, frames_to_draw=frames_to_draw, axis_length=0.15,
                 axis_radius=0.001,)

#visualizer the contact
contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
                                       force_threshold=0.0,
                                       contact_force_scale=15., 
                                       contact_force_radius=0.001,   
                                       plant=plant))
builder.Connect(plant.get_contact_results_output_port(),
                contact_visualizer.GetInputPort("contact_results"))


diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
X_WB = RigidTransform(RotationMatrix(), [0., 0., 0.0334 - 20.0])
plant.SetFreeBodyPose(plant_context, 
                              plant.GetBodyByName("base_link_cracker", cracker_box),
                              X_WB)

meshcat.load()
diagram.Publish(context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [6]:
exp_plant = plant.ToScalarType[Expression]()
exp_plant_context = exp_plant.CreateDefaultContext()
q0 = Variable('q0')
q1 = Variable('q1')
q2 = Variable('q2')
q3 = Variable('q3')
q4 = Variable('q4')
q5 = Variable('q5')
q6 = Variable('q6')
v0 = Variable('v0')
v1 = Variable('v1')
v2 = Variable('v2')
v3 = Variable('v3')
v4 = Variable('v4')
v5 = Variable('v5')

q = np.ndarray((7,1),dtype=object,buffer=np.array([q0,q1,q2,q3,q4,q5,q6]))
v = np.ndarray((6,1),dtype=object,buffer=np.array([v0,v1,v2,v3,v4,v5]))
exp_plant.SetPositions(exp_plant_context, q)
exp_plant.SetVelocities(exp_plant_context, v)

# matrices for the manipulator equations
exp_M = exp_plant.CalcMassMatrixViaInverseDynamics(exp_plant_context)
exp_Cv = exp_plant.CalcBiasTerm(exp_plant_context)
exp_tauG = exp_plant.CalcGravityGeneralizedForces(exp_plant_context)
print("exp_M:", exp_M)
print("exp_Cv:", exp_Cv)
print("exp_tauG:", exp_tauG)

exp_M: [[<Expression "(0.10000000000000001 * (0.0035299999999999984 + 0.013829999999999999 * pow(((q0 * ((2 * q1) / (pow(q0, 2) + pow(q1, 2) + pow(q2, 2) + pow(q3, 2)))) + (q2 * ((2 * q3) / (pow(q0, 2) + pow(q1, 2) + pow(q2, 2) + pow(q3, 2))))), 2) + 0.0074500000000000018 * pow(( - (q0 * ((2 * q2) / (pow(q0, 2) + pow(q1, 2) + pow(q2, 2) + pow(q3, 2)))) + (q1 * ((2 * q3) / (pow(q0, 2) + pow(q1, 2) + pow(q2, 2) + pow(q3, 2))))), 2) + 0.0074500000000000018 * pow(((q0 * ((2 * q3) / (pow(q0, 2) + pow(q1, 2) + pow(q2, 2) + pow(q3, 2)))) + (q1 * ((2 * q2) / (pow(q0, 2) + pow(q1, 2) + pow(q2, 2) + pow(q3, 2))))), 2) + 0.013829999999999999 * pow((1 - (q1 * ((2 * q1) / (pow(q0, 2) + pow(q1, 2) + pow(q2, 2) + pow(q3, 2)))) - (q3 * ((2 * q3) / (pow(q0, 2) + pow(q1, 2) + pow(q2, 2) + pow(q3, 2))))), 2)))">
  <Expression "(0.10000000000000001 * ( - 0.013829999999999999 * (( - (q0 * ((2 * q3) / (pow(q0, 2) + pow(q1, 2) + pow(q2, 2) + pow(q3, 2)))) + (q1 * ((2 * q2) / (pow(q0, 2) + pow(q1, 2) + pow(q2

In [7]:
results = plant.get_contact_results_output_port().Eval(plant_context)
for i in range(results.num_point_pair_contacts()):
    info = results.point_pair_contact_info(i)
    pair = info.point_pair()
    print("depth:",pair.depth,"  id_A:", pair.id_A,"  id_B:", pair.id_B,"  nhat_BA_W:",
          pair.nhat_BA_W, "  p_WCa:",pair.p_WCa, "  p_WCb:",pair.p_WCb)
    print("bodyA_index:",info.bodyA_index(),"  bodyB_index:", info.bodyB_index(),"  contact_force:", info.contact_force(),"  contact_point:",
          info.contact_point(), "  separation_speed:",info.separation_speed(), "  slip_speed:",info.slip_speed())
    print("########################")

In [8]:
query_object = plant.get_geometry_query_input_port().Eval(plant_context)
scene_graph_inspector = query_object.inspector()
signed_distance_pairs = query_object.ComputeSignedDistancePairwiseClosestPoints()
for signed_distance_pair in signed_distance_pairs:
    print("id_A: ", signed_distance_pair.id_A, "   id_B: ", signed_distance_pair.id_B)
    print("p_ACa: ", signed_distance_pair.p_ACa, "   p_BCb: ", signed_distance_pair.p_BCb)
    print("nhat_BA_W: ", signed_distance_pair.nhat_BA_W, "   distance: ", signed_distance_pair.distance)
    print("nhat_BA_W.T: ", np.array([signed_distance_pair.nhat_BA_W]).T*signed_distance_pair.nhat_BA_W)
    print("########################")
    
    geometryA_props = scene_graph_inspector.GetProximityProperties(signed_distance_pair.id_A)
    geometryB_props = scene_graph_inspector.GetProximityProperties(signed_distance_pair.id_B)
    geometryA_friction = geometryA_props.GetProperty("material", "coulomb_friction")
    geometryB_friction = geometryB_props.GetProperty("material", "coulomb_friction")
    combined_friction = pydrake.multibody.plant.CalcContactFrictionFromSurfaceProperties(geometryA_friction, geometryB_friction)
    
    print("id_A_friction: ", geometryA_friction.static_friction(), "   id_B_friction: ", geometryB_friction.static_friction())
    print("combined_friction: ", combined_friction.static_friction(), "   id_B_friction: ", geometryB_friction.static_friction())
    

id_A:  <GeometryId value=4>    id_B:  <GeometryId value=8>
p_ACa:  [ 0.  0. -5.]    p_BCb:  [0.     0.     0.0329]
nhat_BA_W:  [0. 0. 1.]    distance:  9.933699999999998
nhat_BA_W.T:  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]
########################
id_A_friction:  1.0    id_B_friction:  1.0
combined_friction:  1.0    id_B_friction:  1.0
id_A:  <GeometryId value=4>    id_B:  <GeometryId value=16>
p_ACa:  [-0.0795 -0.1042 -5.    ]    p_BCb:  [-0.e+00 -0.e+00  1.e-07]
nhat_BA_W:  [-0. -0.  1.]    distance:  9.9331999
nhat_BA_W.T:  [[ 0.  0. -0.]
 [ 0.  0. -0.]
 [-0. -0.  1.]]
########################
id_A_friction:  1.0    id_B_friction:  1.0
combined_friction:  1.0    id_B_friction:  1.0
id_A:  <GeometryId value=4>    id_B:  <GeometryId value=22>
p_ACa:  [-0.0795  0.1042 -5.    ]    p_BCb:  [-0.e+00 -0.e+00  1.e-07]
nhat_BA_W:  [-0. -0.  1.]    distance:  9.9999999
nhat_BA_W.T:  [[ 0.  0. -0.]
 [ 0.  0. -0.]
 [-0. -0.  1.]]
########################
id_A_friction:  1.0    id_B_friction:  1.0

In [9]:
# normal = np.array([[0, 0, 1]]).T
# print(normal*normal.T)
# print(np.outer(normal,normal))
print(np.inf)

inf


In [10]:
# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a,b):
    return np.array_equal(a, b) and np.array_equal(autoDiffToGradientMatrix(a), autoDiffToGradientMatrix(b))

mu = 0.5 
total_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(cracker_box))
gravity = plant.gravity_field().gravity_vector()
collision_point_frame = [
    plant.GetFrameByName('point_collision5'),
    plant.GetFrameByName('point_collision6'),
    plant.GetFrameByName('point_collision7'),
    plant.GetFrameByName('point_collision8')]

#set contact sequence
N = 100
# in_contact = np.zeros((N))
# in_contact[40:] = 1

T = 1
prog = MathematicalProgram()
# Time steps    
h = prog.NewContinuousVariables(N-1, "h")
prog.AddBoundingBoxConstraint(0.5*T/N, 2.0*T/N, h) 
prog.AddLinearConstraint(sum(h) >= .9*T)
prog.AddLinearConstraint(sum(h) <= 1.1*T)

# Create one context per timestep (to maximize cache hits)
plant_context_list = [plant.CreateDefaultContext() for i in range(N)]
# We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
ad_plant = plant.ToAutoDiffXd()

nq_o = plant.num_positions(cracker_box)
nv_o = plant.num_velocities(cracker_box)
q_o = prog.NewContinuousVariables(nq_o, N, "q_o")
v_o = prog.NewContinuousVariables(nv_o, N, "v_o")
vdot_o = prog.NewContinuousVariables(nv_o, N, "vdot_o")

#initial constrain
q0_o = plant.GetPositions(plant_context, cracker_box)
v0_o = plant.GetVelocities(plant_context, cracker_box)
v0_o[3] = 2.5
prog.AddBoundingBoxConstraint(q0_o, q0_o, q_o[:,0])
prog.AddBoundingBoxConstraint(v0_o, v0_o, v_o[:,0])
prog.AddBoundingBoxConstraint([0]*nv_o, [0]*nv_o, v_o[:,-1])
for n in range(N):
    prog.SetInitialGuess(q_o[:,n], q0_o)  
    # Unit quaternions
    AddUnitQuaternionConstraintOnPlant(plant, q_o[:,n], prog)
    # Running costs:
#     prog.AddQuadraticErrorCost(np.diag([1]*nq_o), q0_o, q_o[:,n])
#     prog.AddQuadraticErrorCost(np.diag([1]*nv_o), [0]*nv_o, v_o[:,n])
#     prog.AddQuadraticErrorCost(np.diag([10]*nv_o), [0]*nv_o, vdot_o[:,n])
#     prog.AddQuadraticErrorCost(np.diag([10]*3), [0]*3, vdot_o[:3,n])
#     prog.AddQuadraticErrorCost(np.diag([10]*2), [0]*2, vdot_o[4:,n])

    
# Make a new autodiff context for this constraint (to maximize cache hits)
ad_plant_context_list = [ad_plant.CreateDefaultContext() for i in range(N)]
def velocity_position_constraint(vars, context_index):
    h, q, v, qn = np.split(vars, [1, 1+nq_o, 1+nq_o+nv_o])
    if isinstance(vars[0], AutoDiffXd):
        if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], q)
        v_from_qdot = ad_plant.MapQDotToVelocity(ad_plant_context_list[context_index], (qn - q)/h)
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index])):
            plant.SetPositions(plant_context_list[context_index], q)
        v_from_qdot = plant.MapQDotToVelocity(plant_context_list[context_index], (qn - q)/h)
    return v - v_from_qdot
for n in range(N-1):
    prog.AddConstraint(
        partial(velocity_position_constraint, context_index=n), 
        lb=[0]*nv_o, ub=[0]*nv_o, 
        vars=np.concatenate(([h[n]], q_o[:,n], v_o[:,n], q_o[:,n+1])))  #(implicit Euler)
    
#accelaration and velocity relation constrain(implicit Euler)
for n in range(N-1):
#     prog.AddConstraint(eq(q_o[:, n+1], q_o[:,n] + h[n]*v_o[:,n]))
    prog.AddConstraint(eq(v_o[:, n+1], v_o[:,n] + h[n]*vdot_o[:,n]))

    
###########add sliding friction complementarity constraint###########start
#relate to world frame
collision_point_contact_force = [prog.NewContinuousVariables(3, N-1, f"collision_point{collision_point}_contact_force") for collision_point in range(4)]
#why only to N-1
for n in range(N-1):
    for collision_point in range(4):
        # Linear friction cone
        prog.AddLinearConstraint(collision_point_contact_force[collision_point][0,n] <= mu*collision_point_contact_force[collision_point][2,n])
        prog.AddLinearConstraint(-collision_point_contact_force[collision_point][0,n] <= mu*collision_point_contact_force[collision_point][2,n])
        prog.AddLinearConstraint(collision_point_contact_force[collision_point][1,n] <= mu*collision_point_contact_force[collision_point][2,n])
        prog.AddLinearConstraint(-collision_point_contact_force[collision_point][1,n] <= mu*collision_point_contact_force[collision_point][2,n])
        # normal force >=0, normal_force == 0 if not in_contact
        prog.AddBoundingBoxConstraint(0, 4*9.81*total_mass, collision_point_contact_force[collision_point][2,n])


# get reference frames for collision_point
ad_collision_point_frame = [
ad_plant.GetFrameByName('point_collision5'),
ad_plant.GetFrameByName('point_collision6'),
ad_plant.GetFrameByName('point_collision7'),
ad_plant.GetFrameByName('point_collision8')]
#relate to world frame
collision_point_contact_force_static = [prog.NewContinuousVariables(3, N-1, f"collision_point{collision_point}_contact_force_static") for collision_point in range(4)]
collision_point_contact_force_sliding = [prog.NewContinuousVariables(3, N-1, f"collision_point{collision_point}_contact_force_sliding") for collision_point in range(4)]
collision_point_c = [prog.NewContinuousVariables(1, N-1, f"collision_point{collision_point}_c") for collision_point in range(4)]

def contact_velocity_complementarity_constraint(vars, context_index):
    q, v, f_static_n, c, f_sliding_t = np.split(vars, [nq_o, nq_o+nv_o, nq_o+nv_o+4, nq_o+nv_o+4+4])
    if isinstance(vars[0], AutoDiffXd):
        if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], q)
        if not autoDiffArrayEqual(v, ad_plant.GetVelocities(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetVelocities(ad_plant_context_list[context_index], v)
        
        #######f_sliding_t[collision_point] is not 2 dimention, is 1 dimension
        print(f_sliding_t[collision_point])
        collision_point_contact_velocity_t = [ad_collision_point_frame[collision_point].CalcSpatialVelocity(ad_plant_context_list[context_index], ad_plant.world_frame(), ad_plant.world_frame()).translational()[:2] for collision_point in range(4)]
        v_t_dot_f_static_n = [collision_point_contact_velocity_t[collision_point] * f_static_n[collision_point] for collision_point in range(4)]
        f_sliding_inverse_direction_v_t = [f_sliding_t[collision_point] + c[collision_point] * collision_point_contact_velocity_t[collision_point] for collision_point in range(4)]
        res = np.concatenate((v_t_dot_f_static_n, f_sliding_inverse_direction_v_t))
        res = np.array(res).flatten()
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index])):
            plant.SetPositions(plant_context_list[context_index], q)
        if not np.array_equal(v, plant.GetVelocities(plant_context_list[context_index])):
            plant.SetVelocities(plant_context_list[context_index], v)
        collision_point_contact_velocity_t = [collision_point_frame[collision_point].CalcSpatialVelocity(plant_context_list[context_index], plant.world_frame(), plant.world_frame()).translational() for collision_point in range(4)]
        v_t_dot_f_static_n = [collision_point_contact_velocity_t[collision_point] * f_static_n[collision_point] for collision_point in range(4)]
        f_sliding_inverse_direction_v_t = [f_sliding_t[collision_point] + c[collision_point] * collision_point_contact_velocity_t[collision_point] for collision_point in range(4)]
        res = np.concatenate((v_t_dot_f_static_n, f_sliding_inverse_direction_v_t))
        res = np.array(res).flatten()
    return res

epsilon = 1e-5
normal = np.array([[0, 0, 1]]).T
for n in range(N-1):
    vars=np.concatenate((q_o[:,n], v_o[:,n],  
            [collision_point_contact_force_static[0][2, n]], [collision_point_contact_force_static[1][2, n]],
            [collision_point_contact_force_static[2][2, n]], [collision_point_contact_force_static[3][2, n]],
            collision_point_c[0][:, n], collision_point_c[1][:, n],
            collision_point_c[2][:, n], collision_point_c[3][:, n],             
            collision_point_contact_force_sliding[0][:2, n], collision_point_contact_force_sliding[1][:2, n],
            collision_point_contact_force_sliding[2][:2, n], collision_point_contact_force_sliding[3][:2, n]))
    prog.AddConstraint(
        partial(contact_velocity_complementarity_constraint, context_index=n), 
        lb=np.concatenate(([-epsilon]*2*4, [0]*2*4)), ub=np.concatenate(([epsilon]*2*4, [0]*2*4)), vars=vars)  
    
    for collision_point in range(4):
        prog.AddLinearConstraint(ge(collision_point_c[collision_point][:, n], 0))
        prog.AddConstraint(eq((collision_point_contact_force_sliding[collision_point][:, n].dot((np.identity(3) - 
                        (mu*mu+1)*normal*normal.T)*collision_point_contact_force_sliding[collision_point][:, n])), 0))
        prog.AddBoundingBoxConstraint(0, 4*9.81*total_mass, collision_point_contact_force_sliding[collision_point][2,n])

        prog.AddLinearConstraint(eq(collision_point_contact_force[collision_point][:, n], 
                                collision_point_contact_force_static[collision_point][:, n] + 
                                   collision_point_contact_force_sliding[collision_point][:, n]))

###########add sliding friction complementarity constraint###########end
        
def manipulator_equations(vars):
    nf = 3
    # split input vector in subvariables
    # configuration, velocity, acceleration, collision_point force
    split_at = [nq_o, nq_o + nv_o, nq_o + 2 * nv_o, nq_o + 2 * nv_o + nf, nq_o + 2 * nv_o + 2 * nf, nq_o + 2 * nv_o + 3 * nf]
    q, v, vdot, f1, f2, f3, f4 = np.split(vars, split_at)
    f = [f1, f2, f3, f4]

    ad_plant_context = ad_plant.CreateDefaultContext()
    ad_plant.SetPositions(ad_plant_context, q)
    ad_plant.SetVelocities(ad_plant_context, v)
    
    # matrices for the manipulator equations
    M = ad_plant.CalcMassMatrixViaInverseDynamics(ad_plant_context)
    Cv = ad_plant.CalcBiasTerm(ad_plant_context)
    tauG = ad_plant.CalcGravityGeneralizedForces(ad_plant_context)

    
    # Jacobian of the collision_point
    # get reference frames for collision_point
    ad_collision_point_frame = [
    ad_plant.GetFrameByName('point_collision5'),
    ad_plant.GetFrameByName('point_collision6'),
    ad_plant.GetFrameByName('point_collision7'),
    ad_plant.GetFrameByName('point_collision8')]

    # compute Jacobian matrix
    J = [ad_plant.CalcJacobianTranslationalVelocity(
        ad_plant_context,
        JacobianWrtVariable.kV,
        ad_collision_point_frame[collision_point],
        [0., 0., 0.],
        ad_plant.world_frame(),
        ad_plant.world_frame())
        for collision_point in range(4)]
    

    tauF = np.zeros((nv_o))
    for collision_point in range(4):
        tauF = tauF + J[collision_point].T.dot(f[collision_point])
        
    
    # return violation of the manipulator equations
    return M.dot(vdot) + Cv - tauG - tauF

# manipulator equations for all t (implicit Euler)
for n in range(N-1):
    vars = np.concatenate((q_o[:, n], v_o[:, n], vdot_o[:, n], 
                           collision_point_contact_force[0][:, n], collision_point_contact_force[1][:, n],
                           collision_point_contact_force[2][:, n], collision_point_contact_force[3][:, n]))
    prog.AddConstraint(manipulator_equations, lb=[0]*nv_o, ub=[0]*nv_o, vars=vars)

In [11]:
snopt = SnoptSolver().solver_id()
prog.SetSolverOption(snopt, 'Iterations Limits', 1e5 if running_as_notebook else 1)
prog.SetSolverOption(snopt, 'Major Iterations Limit', 200 if running_as_notebook else 1)
prog.SetSolverOption(snopt, 'Major Feasibility Tolerance', 5e-6)
prog.SetSolverOption(snopt, 'Major Optimality Tolerance', 1e-4)
prog.SetSolverOption(snopt, 'Superbasics limit', 2000)
prog.SetSolverOption(snopt, 'Linesearch tolerance', 0.9)
#prog.SetSolverOption(snopt, 'Print file', 'snopt.out')



result = Solve(prog)
print(result.get_solver_id().name())
if not result.is_success():
#     print(result.GetInfeasibleConstraintNames(prog))
    print("solve failed!")
else:
    print("solve success")
    q_o_sol = result.GetSolution(q_o)
    print("q_o_sol:", q_o_sol)

AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nderiv=29}
AD{0.0, nder

KeyboardInterrupt: 

In [8]:
v_o_sol = result.GetSolution(v_o)
print("v_o_sol:", v_o_sol)

v_o_sol: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 

In [9]:
vdot_o_sol = result.GetSolution(vdot_o)
print("vdot_o_sol:", vdot_o_sol)

vdot_o_sol: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+0

In [10]:
collision_point_contact_force_sol = [result.GetSolution(collision_point_contact_force[collision_point]) for collision_point in range(4)]

print("collision_point0_contact_force:", collision_point_contact_force_sol[0] 
      + collision_point_contact_force_sol[1] 
      + collision_point_contact_force_sol[2] 
      + collision_point_contact_force_sol[3]
     )


collision_point0_contact_force: [[-6.58966793e-01 -6.05886075e-01 -7.59043020e-01 -5.60294423e-01
  -2.46481743e-01 -4.00795163e-01 -4.76118088e-01 -9.02213562e-01
  -2.08024001e-01 -3.31389928e-01 -3.35789290e-01 -2.79890715e-01
  -1.98053085e-01 -2.19410040e-01 -2.15260149e-01 -2.12102949e-01
  -2.09968900e-01 -2.06902919e-01 -2.03873155e-01 -2.00883052e-01
  -1.97936047e-01 -1.95022109e-01 -1.91698897e-01 -1.84298366e-01
  -2.40011110e-01 -3.09383439e-01 -2.75360005e-01 -1.77741898e-01
  -1.78605450e-01 -1.67717919e-01 -1.79419946e-01 -1.79914604e-01
  -1.80433941e-01 -1.80977465e-01 -1.81544684e-01 -1.82098062e-01
  -1.83472324e-01 -1.90846611e-01 -1.80922716e-01 -1.91426287e-01
  -1.87103565e-01 -1.90701251e-01 -1.81394422e-01 -7.85840322e-01
  -1.80341644e-01 -1.85426003e-01 -1.85827106e-01 -1.86620239e-01
  -1.86531955e-01 -1.86405468e-01 -1.39183379e-01 -1.27515732e-01
  -1.27264265e-01 -1.27013558e-01 -1.26763277e-01 -1.26513455e-01
  -1.85474546e-01 -1.26075511e-01 -1.2573275

In [11]:
collision_point_contact_force_static_sol = [result.GetSolution(collision_point_contact_force_static[collision_point]) for collision_point in range(4)]

print("collision_point_contact_force_static:", collision_point_contact_force_static_sol[0][2] 
#       + collision_point_contact_force_static_sol[1] 
#       + collision_point_contact_force_static_sol[2] 
#       +collision_point_contact_force_static_sol[3]
     )


NameError: name 'collision_point_contact_force_static' is not defined

In [20]:
collision_point_contact_force_sliding_sol = [result.GetSolution(collision_point_contact_force_sliding[collision_point]) for collision_point in range(4)]

print("collision_point_contact_force_sliding:", collision_point_contact_force_sliding_sol[3][2] 
#       + collision_point_contact_force_sliding_sol[1] 
#       + collision_point_contact_force_sliding_sol[2] 
#       + collision_point_contact_force_sliding_sol[3]
     )


collision_point_contact_force_sliding: [ 1.98999248e-05  2.49871327e-05  1.11964508e-05  1.19581974e-05
  2.35260524e-05  2.17745278e-05  3.18984794e-05  5.12230637e-04
  1.14333011e-04  3.56874266e-06  4.19693177e-05  1.41743162e-05
  2.31112937e-05  7.09124898e-05  3.93301465e-05  6.48158523e-05
  6.26275024e-05  5.42892568e-05  1.20383568e-05  2.39230971e-04
  0.00000000e+00  1.52948256e-05  0.00000000e+00  1.21199302e-05
  1.80546246e-05  5.58131342e-07  4.60056399e-06  0.00000000e+00
  0.00000000e+00  3.93270696e-05  2.40587348e-04  0.00000000e+00
  8.51122893e-05  7.96135591e-07  1.39815317e-05  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  8.06498895e-05
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7.30638421e-06  7.34396787e-06  0.00000000e+00  1.22841482e-07
  0.00000000e+00  3.07010297e-05  2.16814285e-05  0.00000000e+00
  0.00000000e+00  6.72629554e-05  0.00000000e+00  5.64969938e-05
  1.21481535e-05  0.00000000e+00  3.24146165e-05  2

In [21]:
collision_point_c_sol = [result.GetSolution(collision_point_c[collision_point]) for collision_point in range(4)]

print("collision_point_c:", collision_point_c_sol
     )

collision_point_c: [array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [12]:
# manipulator equations for all t (implicit Euler)
for n in range(N-1):
    vars = np.concatenate((q_o_sol[:, n], v_o_sol[:, n], vdot_o_sol[:, n], 
                           collision_point_contact_force_sol[0][:, n], collision_point_contact_force_sol[1][:, n],
                           collision_point_contact_force_sol[2][:, n], collision_point_contact_force_sol[3][:, n]))
    print("manipulator_equations:", autoDiffToValueMatrix(manipulator_equations(vars)))

manipulator_equations: [[ 2.43937866e-16]
 [ 1.83559663e-16]
 [ 1.33204401e-17]
 [ 9.71445147e-17]
 [ 6.24500451e-17]
 [-2.44249065e-15]]
manipulator_equations: [[ 4.11476409e-15]
 [-6.60929644e-15]
 [ 7.25808302e-15]
 [-8.32667268e-17]
 [ 0.00000000e+00]
 [-3.33066907e-16]]
manipulator_equations: [[-5.26488575e-16]
 [-3.53883589e-16]
 [ 7.37257477e-18]
 [-9.89208715e-14]
 [ 4.85722573e-17]
 [-1.11022302e-16]]
manipulator_equations: [[ 4.12170298e-15]
 [-6.59541866e-15]
 [ 7.26935873e-15]
 [ 1.11022302e-16]
 [ 2.77555756e-17]
 [ 2.22044605e-16]]
manipulator_equations: [[ 4.11476409e-15]
 [-6.59194921e-15]
 [ 7.26241983e-15]
 [-1.11022302e-16]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
manipulator_equations: [[ 4.10782519e-15]
 [-6.58501031e-15]
 [ 7.25981775e-15]
 [-2.77555756e-17]
 [ 0.00000000e+00]
 [ 5.55111512e-16]]
manipulator_equations: [[ 4.12864187e-15]
 [-6.61276589e-15]
 [ 7.26935873e-15]
 [ 1.38777878e-16]
 [-2.77555756e-17]
 [ 1.11022302e-16]]
manipulator_equations: [[ 4.121702

In [16]:
print(q0_o)
print(v0_o)

[1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]


In [19]:
def manipulator_equations_val(vars):
    nf = 3
    # split input vector in subvariables
    # configuration, velocity, acceleration, collision_point force
    split_at = [nq_o, nq_o + nv_o, nq_o + 2 * nv_o, nq_o + 2 * nv_o + nf, nq_o + 2 * nv_o + 2 * nf, nq_o + 2 * nv_o + 3 * nf]
    q, v, vdot, f1, f2, f3, f4 = np.split(vars, split_at)
    f = [f1, f2, f3, f4]

    plant_context = plant.CreateDefaultContext()
    plant.SetPositions(plant_context, q)
    plant.SetVelocities(plant_context, v)
    print("v:\n" , v)
    # matrices for the manipulator equations
    M = plant.CalcMassMatrixViaInverseDynamics(plant_context)
    Cv = plant.CalcBiasTerm(plant_context)
    tauG = plant.CalcGravityGeneralizedForces(plant_context)
#     print("tauG:\n" , tauG)
#     print("g:\n" , plant.gravity_field().gravity_vector())
#     print("Cv:\n" , Cv)
    # Jacobian of the collision_point
    # get reference frames for collision_point
    collision_point_frame = [
    plant.GetFrameByName('point_collision5'),
    plant.GetFrameByName('point_collision6'),
    plant.GetFrameByName('point_collision7'),
    plant.GetFrameByName('point_collision8')]

    # compute Jacobian matrix
    J = [plant.CalcJacobianTranslationalVelocity(
        plant_context,
        JacobianWrtVariable.kV,
        collision_point_frame[collision_point],
        [0., 0., 0.],
        plant.world_frame(),
        plant.world_frame())
        for collision_point in range(4)]
    

    tauF = np.zeros((nv_o))
    for collision_point in range(4):
        tauF = tauF + J[collision_point].T.dot(f[collision_point])
#         print("J", collision_point, ": \n" , J[collision_point])
        
#     print("#######################")
    # return violation of the manipulator equations
    return M.dot(vdot) + Cv - tauG - tauF

In [20]:
for n in range(N-1):
    vars = np.concatenate((q_o_sol[:, n], v_o_sol[:, n], vdot_o_sol[:, n], 
                           collision_point_contact_force_sol[0][:, n], collision_point_contact_force_sol[1][:, n],
                           collision_point_contact_force_sol[2][:, n], collision_point_contact_force_sol[3][:, n]))
#     print("manipulator_equations:", manipulator_equations_val(vars))
    manipulator_equations_val(vars)

v:
 [0.  0.  0.  2.5 0.  0. ]
v:
 [ 2.98143223e-07 -3.68686061e-07  1.02814525e-07  2.50000416e+00
  0.00000000e+00 -1.15117830e-06]
v:
 [ 4.56233430e-07 -5.39370011e-07  1.46652064e-07  2.50000478e+00
 -3.29378172e-09 -1.34869422e-06]
v:
 [ 5.30275379e-07 -7.28292280e-07  1.65565986e-07  2.50000525e+00
 -6.49156300e-09 -1.46332445e-06]
v:
 [ 6.34758557e-07 -1.10915379e-06  1.85201742e-07  2.50000598e+00
 -9.24843485e-09 -1.65096781e-06]
v:
 [ 7.30832950e-07 -1.43951098e-06  2.02670452e-07  2.50000665e+00
 -1.15763081e-08 -1.81040578e-06]
v:
 [ 8.15051196e-07 -1.71609707e-06  2.17654869e-07  2.50000727e+00
 -1.35529967e-08 -1.94531287e-06]
v:
 [ 1.09907655e-06 -2.29191835e-06  3.05469279e-07  2.50000838e+00
 -2.30038150e-08 -2.35408226e-06]
v:
 [ 1.91370217e-06 -3.61670086e-06  5.61205298e-07  2.50000998e+00
 -4.08216024e-08 -4.09691797e-06]
v:
 [ 2.73569103e-06 -4.87102540e-06  8.16034419e-07  2.50001152e+00
 -5.80171465e-08 -5.85474608e-06]
v:
 [ 5.31697004e-06 -4.13379252e-06  1.003

In [19]:
p_WF = plant.CalcPointsPositions(plant_context, plant.GetFrameByName('point_collision5'), [0,0,0], plant.world_frame())
print(p_WF)
X_WF = plant.CalcRelativeTransform(plant_context, plant.GetFrameByName('point_collision5'), plant.world_frame())
print(X_WF)

[[0.]
 [0.]
 [0.]]
RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)


In [19]:
print(gravity)

[ 0.    0.   -9.81]
